This exercise will require you to pull some data from the Quandl API. Quandl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'REDACTED'

Quandl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import collections

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2019-07-23&end_date=2019-07-23?api_key=' + API_KEY
r = requests.get(url)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.json()

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (2019-07-10) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2019-07-24T22:18:57.969Z',
  'newest_available_date': '2019-07-24',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2019-07-23',
  'end_date': '2019-07-23',
  'data': [['2019-07-23',
    97.2,
    98.2,
    97.05,
    98.1,
    None,
    91724.0,
    8971253.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2017-01-01&end_date=2017-12-31?api_key=' + API_KEY
r = requests.get(url)

In [6]:
afx = r.json()
afx['dataset']['column_names']

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [7]:
data = afx['dataset']['data']
data_dict = {}

for d in data:
    day = d[0]
    
    day_dict = {}
    day_dict['Open'] = d[1]
    day_dict['High'] = d[2]
    day_dict['Low'] = d[3]
    day_dict['Close'] = d[4]
    day_dict['Change'] = d[5]
    day_dict['Traded Volume'] = d[6]
    day_dict['Turnover'] = d[7]
    day_dict['Last Price of the Day'] = d[8]
    day_dict['Daily Traded Units'] = d[9]
    day_dict['Daily Turnover'] = d[10]
    
    data_dict[day] = day_dict

In [8]:
highest_opening_price = 0.0
lowest_opening_price = float('inf')

for day, data in data_dict.items():
    if data['Open'] is None:
        pass
    elif data['Open'] > highest_opening_price:
        highest_opening_price = data['Open']
    elif data['Open'] < lowest_opening_price:
        lowest_opening_price = data['Open']
        
print(f'The highest opening price for 2017 was ${highest_opening_price}.')
print(f'The lowest opening price for 2017 was ${lowest_opening_price:.2f}.')

The highest opening price for 2017 was $53.11.
The lowest opening price for 2017 was $34.00.


In [9]:
largest_change = 0.0

for day, data in data_dict.items():
    change = data['High'] - data['Low']
    
    if change > largest_change:
        largest_change = change
        
print(f'The largest change in any one day was ${largest_change:.2f}.')

The largest change in any one day was $2.81.


In [10]:
largest_change = 0.0

for day, data in data_dict.items():
    if day == '2017-12-29':
        prev_day_closing = data['Close']
        
    change = abs(data['Close'] - prev_day_closing)
    if change > largest_change:
        largest_change = change
        
    prev_day_closing = data['Close']
    
print(f'The largest change between any two days was ${largest_change:.2f}.')

The largest change between any two days was $2.56.


In [11]:
total_trading_volume = 0.0

for day, data in data_dict.items():
    total_trading_volume += data['Traded Volume']
    
average_trading_volume = total_trading_volume / len(data_dict)
print(f'The average daily trading volume for 2017 was ${average_trading_volume:.2f}.')

The average daily trading volume for 2017 was $89124.34.


In [12]:
def median(data: list):
    sorted_data = sorted(data)
    length = len(sorted_data)
    mid = length / 2

    # If even, get two middle values
    if length % 2 == 0:
        # Subtract 1 from half of length for first middle value since 0-based indexing
        median = (sorted_data[int(mid) - 1] + sorted_data[int(mid)]) / 2
    else:
        # Use int instead of round for floor value since 0-based indexing
        median = sorted_data[int(mid)]
        
    return median


trading_volumes = []

for day, data in data_dict.items():
    trading_volumes.append(data['Traded Volume'])
    
print(f'The median trading volume for 2017 was ${median(trading_volumes):.2f}.')

The median trading volume for 2017 was $76286.00.
